In [87]:
#IMPORT DE BIBLIOTECAS

from collections import defaultdict
import pandas as pd
import numpy as np
import os

from scipy.sparse import csr_matrix

from matplotlib import pyplot as plt
import seaborn as sns

from surprise import Dataset, Reader, SVD, SVDpp
from surprise.model_selection import train_test_split, cross_validate
from surprise import accuracy
from surprise.model_selection import cross_validate

import pickle
import flask

In [84]:
df_de_cat = pd.read_csv('category_de.txt') # DataFrame Informativo das categorias

In [3]:
clicks_de = pd.read_csv('clicks_de_sample_2.txt', sep = ',', header=0) # df de clicks

In [95]:
clicks_de.head()

,UserId,OfferId,OfferViewId,CountryCode,Category,Source,UtcDate,OfferTitle,Cat_clicks,User,Offer,Ancestor_ID,Name,UserClicks
0,d37a4e15a65cb9e6b2a5c3bd6b109c2cbb8ea28ea0e249...,874006e61139f2b56ba21a46fe554753,874006e61139f2b56ba21a46fe554753-1076982502353...,de,128101,f7151fb3a6269230dcaa048646568c53ad69ab3b3d24ae...,2016-06-01 13:24:13.0,Buch - Süßer die Glocken...,3,0,0,128101 128001 100346123 601,Erotikliteratur,1
1,67d65125918799e0cb955526f8245b633c13729f20644d...,568777b0316aafe2b858d9a10e9ead28,568777b0316aafe2b858d9a10e9ead28-1076982502353...,de,128101,f7151fb3a6269230dcaa048646568c53ad69ab3b3d24ae...,2016-06-01 03:32:14.0,Buch - Shades of Grey 1 - Geheimes Verlangen,3,1,1,128101 128001 100346123 601,Erotikliteratur,1
2,7ff9936d1bc8fd24ff1e7333e0ff0b70b5214a949f3aa3...,874006e61139f2b56ba21a46fe554753,874006e61139f2b56ba21a46fe554753-1076982502353...,de,128101,f7151fb3a6269230dcaa048646568c53ad69ab3b3d24ae...,2016-06-01 07:58:40.0,Buch - Süßer die Glocken...,3,2,0,128101 128001 100346123 601,Erotikliteratur,1
3,e94d7bb5f915bcc3a5968f6454d9dcd7fbec5885e09286...,e30f11660b6d8559e66bb00b9847324f,e30f11660b6d8559e66bb00b9847324f-1076982042286...,de,100270123,7ffbeedfb14f46f259001cefc4d79048ccef25b9e87207...,2016-06-02 02:26:14.0,Masturbator Mood Pleasure Stroker UR3 Thick Ri...,7,3,2,100270123 128001 100346123 601,Sex Gags,1
4,515408cbdab92663d58bc03e6d14bcb298250f89052d99...,7e38a7b5c2881d5b38fada633fc36507,7e38a7b5c2881d5b38fada633fc36507-1076982511973...,de,100270123,978f3cfa5eec8581696bf86dfd458583ad718be08831ba...,2016-06-01 21:51:30.0,Masturbator Hustler Porn Star Pussy and Ass St...,7,4,3,100270123 128001 100346123 601,Sex Gags,8


In [4]:
# CRIAÇÃO DE COLUNA DE CLICKS POR CATEGORIA
clicks_de.drop(columns = ['Keywords'], axis = 1, inplace=True)
#clicks_de['Category'] = clicks_de['Category'].astype(str)

pontuacao_de = pd.DataFrame(clicks_de['Category'].value_counts()).reset_index()
pontuacao_de.columns = ['Category', 'Pontuacao']
dici = dict(zip(pontuacao_de.Category, pontuacao_de.Pontuacao))

clicks_de['Cat_clicks'] = clicks_de['Category'].map(dici)
clicks_de.sort_values(by='Cat_clicks', inplace = True)

In [5]:
## RENOMEANDO OFERTAS E USUÁRIOS
clicks_de['User'] = clicks_de['UserId'].map(dict(zip(clicks_de['UserId'].unique(), 
                                                                        range(len(clicks_de['UserId']))))) # Redefinindo indices para usuário apenas como numérico

clicks_de['Offer'] = clicks_de['OfferTitle'].map(dict(zip(clicks_de['OfferTitle'].unique(), 
                                                                        range(len(clicks_de['OfferTitle']))))) #  Redefinindo indices para ofertas apenas como numérico considerando o OfferTitle

In [6]:
clicks_de = clicks_de.merge(df_de_cat, left_on = 'Category', right_on = 'ID').drop('ID', axis = 1) 

In [7]:
clicks_per_user_product = clicks_de.groupby(by=['UserId','OfferId']).count()[['Name']].reset_index()

In [8]:
clicks_per_user_product['User'] = clicks_per_user_product['UserId'].map(dict(zip(clicks_per_user_product['UserId'].unique(), 
                                                                        range(len(clicks_per_user_product['UserId']))))) # Redefinindo indices para usuário apenas como numérico


clicks_per_user_product['Offer'] = clicks_per_user_product['OfferId'].map(dict(zip(clicks_per_user_product['OfferId'].unique(), 
                                                                        range(len(clicks_per_user_product['OfferId']))))) #  Redefinindo indices para ofertas apenas como numérico considerando o OfferTitle

In [9]:
clicks_de['UserClicks'] = clicks_de['UserId'].map(dict(zip(clicks_de['UserId'].value_counts().index,
                                                           clicks_de['UserId'].value_counts().values))) #Criação de coluna com nº total de cliques do usuário

In [10]:
#Cap minimo de clicks para integrar o sistema de recomendação
clicks_de_filtered = clicks_de[(clicks_de.UserClicks > 40)]

#DataFrame de base para utilização no Surprise
user_item_clicks = clicks_de_filtered[['User','Offer','UserClicks']]

In [11]:
log = user_item_clicks.groupby(by=['User','Offer']).count().reset_index()
log['UserClicks'] = 1
display(log.head())

,User,Offer,UserClicks
0,249,83,1
1,249,3481,1
2,249,7628,1
3,249,7708,1
4,249,9575,1


In [12]:
#Matriz esparsa
pivo_log = log.pivot_table(columns = 'Offer', index = 'User', values = 'UserClicks').fillna(0)

In [13]:
#Nova relação de clicks em cada produto por usuário a partir da matriz esparsa para uso no Surprise
pivo_log_melted = pd.melt(frame = pivo_log, var_name = 'Products', value_name = 'Clicked?', ignore_index = False).reset_index()

## TESTES SURPRISE

In [15]:
#Escala de avaliação
reader = Reader(rating_scale = (0,1))

#Criação de objeto Dataset do Surprise
data = Dataset.load_from_df(df = pivo_log_melted, reader=reader)

In [79]:
#Criação do modelo e teste por validação cruzada
svd = SVD() # Falta otimização de parametros e testagem de outros modelos
cross_validate(svd, data, measures = ['RMSE','MAE'], cv = 5)

In [80]:
svdpp = SVDpp() # Falta otimização de parametros e testagem de outros modelos
cross_validate(svd, data, measures = ['RMSE','MAE'], cv = 5)

{'test_rmse': array([0.13410884, 0.13147261, 0.13026821, 0.13226135, 0.13404056]),
 'test_mae': array([0.03655328, 0.03604339, 0.03574242, 0.03631242, 0.03567607]),
 'fit_time': (10.09976577758789,
  10.098920106887817,
  10.156632900238037,
  10.116280794143677,
  10.137130498886108),
 'test_time': (0.3370988368988037,
  0.4418525695800781,
  0.4474148750305176,
  0.44689202308654785,
  0.32556772232055664)}

In [85]:
#Criação do set de treino com o dataset completo
trainsetfull = data.build_full_trainset()

#Fit do modelo com o set de treino
svd.fit(trainsetfull)
svdpp.fit(trainsetfull)

In [86]:
svdpp.fit(trainsetfull)

In [147]:
#Fetch de informação de usuários pra teste
pivo_log_melted

,User,Products,Clicked?
0,249,83,1.0
1,362,83,0.0
2,567,83,0.0
3,815,83,0.0
4,919,83,0.0
...,...,...,...
195635,293680,292597,0.0
195636,342625,292597,0.0
195637,398935,292597,1.0
195638,398983,292597,0.0


In [90]:
#Teste de predição com usuário específico frente a um produto específico
prediction = svdpp.predict(uid = 815, iid = 282283)

In [91]:
prediction

Prediction(uid=815, iid=282283, r_ui=None, est=0.15451328087152685, details={'was_impossible': False})

# FUNÇÃO DE RECOMENDAÇÕES

In [181]:
###########TESTES###############
#np.nonzero(pivo_log_melted[pivo_log_melted['User']==user_id]['Clicked?'].to_numpy() == 0)
list_of_unclicked_items = pivo_log_melted[(pivo_log_melted['User']==293680) & (pivo_log_melted['Clicked?']==0)]['Products']
#user_set = [[249, item_id, 0] for item_id in list_of_unclicked_items]

In [183]:
list_of_unclicked_items.map(offer_title)

68        Black Magic Design - Intensity Pro Video PCIe ...
141       BIOMARIS GmbH & Co. KG BIOMARIS SUPER RICH M PARF
214           X4-TECH Kinder CD-Player Bobby Joey inkl. MP3
287       X4-TECH Kinder CD-Player Bobby Joey mit Anti-S...
360               Happy Lola Sexy Pasties Mix Pack - 8 Stk.
                                ...                        
195343    Opel Zafira Tourer 1.8 SELECTION - Business Paket
195416    Volkswagen Golf V Tour ALU BRE ZAHNr KD TÜV NE...
195489    Opel Insignia 1.4 T ST XENON KAMERA NAVI AGR O...
195562      Ford Granada 2.8 Ghia *Klimaanlage - AutomatiK*
195635    Mercedes-Benz CLS 500 Shooting Brake 4M BE AMG...
Name: Products, Length: 2652, dtype: object

In [235]:
##Função que retorna as top-N recomendações para cada usuário de um conjunto de predições

def recommendations_from_SVDpp(user_id, pivo_log, algo, n_recommendations):
    
    # determine list of unseen itemns by user_id
    list_of_unclicked_items = pivo_log[(pivo_log['User']==user_id) & (pivo_log['Clicked?']==0)]['Products']
    
    # set up user set with unrated movies
    user_set = [[user_id, item_id, 0] for item_id in list_of_unclicked_items]
    
    # generate predictions
    predictions = algo.test(user_set)

    top_n_recommendations = defaultdict(list)
    
    for user_id, item_id, _, est, _ in predictions:

        top_n_recommendations[user_id].append((item_id, est))

        
    for user_id, ratings in top_n_recommendations.items():

        ratings.sort(key=lambda x: x[1], reverse=True)

        top_n_recommendations[user_id] = ratings[:n_recommendations]


    count = 0

    print("Recommendations for user with id {}: ".format(user_id))

    message = ''

    for item_index, score in top_n_recommendations[user_id]: 

        count +=1

        message += (str(count)+'. '+str(offer_title[item_index])+' predicted rating = '+str(round(score,3)))
        message +='\n'

    return message

In [236]:
recommendations_from_SVDpp(293680, pivo_log_melted, svdpp, 5)

Recommendations for user with id 293680: 


'1. Renault Laguna II Exception predicted rating = 0.209\n2. Seat Leon Cupra R 2.0 T FSI Navi predicted rating = 0.195\n3. Audi Q5 2.0 TDI QUATTRO LEDER NAVI+ XENON+ predicted rating = 0.195\n4. Subaru WRX STI Sport TOP neuwertig predicted rating = 0.194\n5. Mercedes-Benz A 140 KLIMAANLAGE predicted rating = 0.19\n'

In [188]:
recommendations_from_SVDpp(293680, pivo_log_melted, svd, 5)

Recommendations for user with id 293680: 
1. Renault Laguna II Exception, predicted rating = 0.212
2. Seat Leon Cupra R 2.0 T FSI Navi, predicted rating = 0.198
3. Audi Q5 2.0 TDI QUATTRO LEDER NAVI+ XENON+, predicted rating = 0.196
4. Mercedes-Benz A 140 KLIMAANLAGE, predicted rating = 0.192
5. Subaru WRX STI Sport TOP neuwertig, predicted rating = 0.188


In [180]:
pivo_log_melted[(pivo_log_melted['User']==293680) & pivo_log_melted['Clicked?']==1]['Products'].map(offer_title).values

array(['Samsung Galaxy S5 mini blau', 'Samsung Galaxy S III Neo blau',
       'Samsung Galaxy S6 edge Schwarz 64 GB',
       'Samsung Galaxy S3 Neo I9301 Onyx Black',
       'Samsung Galaxy S5 mini weiÃŸ', 'Samsung Galaxy S III Neo weiÃŸ',
       'Samsung Galaxy S5 Weiß 16GB', 'Samsung Galaxy S5 mini schwarz',
       'Samsung Galaxy Note 4 N910F 32GB Charcoal Black',
       'Samsung Galaxy A5 WeiÃŸ', 'Huawei P8lite schwarz',
       'Samsung Galaxy S7 Edge 32 GB Schwarz',
       'Apple iPhone 5s 32 GB Silber', 'Samsung Galaxy S5 neo Schwarz',
       'Samsung Galaxy S5 mini SM-G800F Shimmery-White',
       'Samsung Galaxy S3 Neo I9301 Metallic Blue',
       'simvalley Mobile Premium-Notruf-Handy XL-959 mit Dual-SIM Vertragsfrei',
       'Samsung Galaxy S5 Neo gold', 'Nokia 215 Dual SIM White',
       'Samsung Galaxy S5 mini SM-G800F Charcoal-Black',
       'Samsung Galaxy S7 Edge SM-G935F 32GB Black Onyx',
       'iPhone 5s gebraucht - simlockfrei Gold 16GB Zustand C',
       'Samsung Ga

In [ ]:
# Than predict ratings for all pairs (u, i) that are NOT in the training set.
testset = trainset.build_anti_testset()
predictions = algo.test(testset)

top_n = get_top_n(predictions, n=10)

# Print the recommended items for each user
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])


An item is considered relevant if its true rating rui is greater than a given threshold. An item is considered recommended if its estimated rating r^ui is greater than the threshold, and if it is among the k highest estimated ratings.

In [ ]:
def precision_recall_at_k(predictions, k=10, threshold=0.5):
    """Return precision and recall at k metrics for each user"""

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in user_ratings[:k])

        # Precision@K: Proportion of recommended items that are relevant
        # When n_rec_k is 0, Precision is undefined. We here set it to 0.

        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0

        # Recall@K: Proportion of relevant items that are recommended
        # When n_rel is 0, Recall is undefined. We here set it to 0.

        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0

    return precisions, recalls

In [159]:
#Dicionário de ofertas
offer_title = dict(zip(clicks_de_filtered['Offer'].unique(), clicks_de_filtered['OfferTitle'].unique())) #Criação do dicionário que relaciona o nome da oferta (OfferTitle) com o código #Offer para tradução da recomendação dada

### Salvar os modelos

In [89]:
pickle.dump(svd, open('svd.pkl', 'wb'))
pickle.dump(svdpp, open('svdpp.pkl', 'wb'))

In [192]:
pivo_log_melted.to_csv('sparse_melt.csv', index = False)

In [197]:
model = pickle.load(open('C:/Users/gabri/Desktop/DH/PI/svdpp.pkl', 
                         'rb'))

In [198]:
import json

In [206]:
#json.dump(offer_title, open('Ofertas.json', 'w'))
d = {int(k): v for k, v in offer_title.items()}

In [219]:
pickle.dump(offer_title, open("offers.pkl", "wb"))

In [220]:
output = pickle.load(open('offers.pkl', 'rb'))

In [229]:
(str(count)+'. '+str(offer_title[item_index])+' predicted rating = '+str(round(score,3)))

'0. Black Magic Design - Intensity Pro Video PCIe Card predicted rating = 20.34'

In [228]:
count=0
str(count)

'0'